<div style="text-align:center; font-family:Tahoma, Arial; line-height:1.8;">

  <div style="font-size:42px; font-weight:bold; color:#0F8298;">
    machine learning 14
  </div>

  <div style="font-size:28px; font-weight:600; color:#6C3BAA; margin-top:8px;">
    Recommender System </div>
  

  <div style="font-size:18px; color:#4b4f9c;">
</div>
Collaborative Filtering

فیلتر کردن مشارکتی

<hr style="width:60%; margin:20px auto; border:1px solid #ddd;">

<font color=497890 size=3>

اهداف: پس از اتمام این آزمایشگاه شما قادر خواهید بود

سیستم پیشنهاددهنده بر اساس فیلترسازی مشارکتی ایجاد کنید

<font color=GREEN size=5>
HAKAN Fatemi (www.hooko.ir)


____
</div> </div>

سیستم‌های پیشنهاددهنده مجموعه‌ای از الگوریتم‌ها هستند که برای توصیه آیتم‌ها به کاربران بر اساس اطلاعات گرفته‌شده از کاربر استفاده می‌شوند. این سیستم‌ها همه‌جا حاضر شده‌اند و معمولاً در فروشگاه‌های آنلاین، پایگاه‌های داده فیلم و جستجوگرهای شغل دیده می‌شوند. در این نوتبوک، سیستم‌های پیشنهاددهنده مبتنی بر **فیلتر کردن مشارکتی** را بررسی کرده و یک نسخه ساده از آن را با استفاده از پایتون و کتابخانه $Pandas$ پیاده‌سازی می‌کنیم.

<h1>فهرست مطالب</h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#ref1">دریافت داده‌ها</a></li>
        <li><a href="#ref2">پیش‌پردازش</a></li>
        <li><a href="#ref3">فیلتر کردن مشارکتی</a></li>
    </ol>
</div>
<br>
<hr>


<a id="ref1"></a>


# دریافت داده‌ها

این دوسلول کد فایل زیپ را دانلود میکند و دو فایل $CSV$ را استخراج میکند و در ‌پوشه‌ی همین کد ذخیره میکند

توجه: بعد از استخراج فایل زیپ را پاک میکند و تنها دو فایل دیتا را نگه میدارد

In [158]:
import os
import requests

if all(os.path.exists(f) for f in ["movies.csv", "ratings.csv"]):
    print(f"{", ".join(["movies.csv", "ratings.csv"])} already exist. No action taken.")
else:
    url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/data/moviedataset.zip"
    response = requests.get(url)
    with open("moviedataset.zip", "wb") as f:
        f.write(response.content)
    print("File downloaded successfully.")


movies.csv, ratings.csv already exist. No action taken.


In [159]:
import os
import zipfile
import shutil

if all(os.path.exists(f) for f in ["movies.csv", "ratings.csv"]):
    print(f"{", ".join(["movies.csv", "ratings.csv"])} already exist. No action taken.")
else:
    with zipfile.ZipFile("moviedataset.zip", "r") as zip_ref:
        zip_ref.extractall(".")
    print("Files extracted successfully.")

    for file_name in ["movies.csv", "ratings.csv"]:
        source_path = f"ml-latest/{file_name}"
        destination_path = file_name
        shutil.copy(source_path, destination_path)
        print(f"File copied to {destination_path} successfully.")

    os.remove("moviedataset.zip")
    shutil.rmtree("ml-latest")
    print("ZIP file and extracted folder deleted successfully.")


movies.csv, ratings.csv already exist. No action taken.


حالا شما آماده شروع کار با داده‌ها هستید

<hr>

<a id="ref2"></a>

# Preprocessing


اول از همه، بیایید تمام کتابخانه‌ها $(imports)$ را وارد کنیم

In [160]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

حال بیایید هر فایل را در $DataFrame$ مربوطه بخوانیم

In [161]:
# ذخیره اطلاعات فیلم در یک دیتافریم پانداس
movies_df = pd.read_csv("movies.csv")
# ذخیره اطلاعات کاربر در یک دیتافریم پانداس
ratings_df = pd.read_csv("ratings.csv")

بیایید همچنین نگاهی بیندازیم که هر کدام از آن‌ها چگونه سازمان‌دهی شده‌اند

In [162]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


بنابراین هر فیلم دارای یک شناسه یکتا، یک عنوان همراه با سال انتشار آن (که ممکن است حاوی کاراکترهای یونیکد باشد) و چندین ژانر مختلف در همان فیلد است. بیایید سال را از ستون عنوان حذف کرده و آن را در ستون جداگانه‌ای قرار دهیم با استفاده از تابع کاربردی [$extract$](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.extract.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2021-01-01#pandas.Series.str.extract) که پانداس دارد

بیایید سال را از ستون **عنوان** با استفاده از تابع $replace$ پانداس حذف کرده و در یک ستون جدید به نام **سال** ذخیره کنیم.

In [163]:
# استفاده از عبارات منظم برای یافتن سال ذخیره شده بین پرانتزها
# پرانتزها را مشخص می‌کنیم تا با فیلم‌هایی که سال در عنوانشان دارند تداخل نداشته باشیم
movies_df["year"] = movies_df.title.str.extract("(\(\d\d\d\d\))",expand=False)
# حذف پرانتزها
movies_df["year"] = movies_df.year.str.extract("(\d\d\d\d)",expand=False)
# حذف سال‌ها از ستون عنوان
movies_df["title"] = movies_df.title.str.replace("(\(\d\d\d\d\))", "", regex=True)
# برای خلاص شدن از هر کاراکتر فضای خالی انتهایی که ممکن است ظاهر شده باشد strip اعمال تابع 
movies_df["title"] = movies_df["title"].apply(lambda x: x.strip())

<>:3: SyntaxWarning: "\(" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\("? A raw string is also an option.
<>:5: SyntaxWarning: "\d" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\d"? A raw string is also an option.
<>:7: SyntaxWarning: "\(" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\("? A raw string is also an option.
<>:3: SyntaxWarning: "\(" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\("? A raw string is also an option.
<>:5: SyntaxWarning: "\d" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\d"? A raw string is also an option.
<>:7: SyntaxWarning: "\(" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\("? A raw string is also an option.
C:\Users\mrb\AppData\Local\Temp\ipykernel_12616\4029105623.py:3: SyntaxWarning: "\

بیایید به نتیجه نگاهی بیندازیم

In [164]:
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


با این حال، بیایید ستون ژانرها را نیز حذف کنیم چون برای این سیستم پیشنهاددهنده خاص به آن نیاز نخواهیم داشت

In [165]:
# حذف ستون ژانرها
movies_df = movies_df.drop("genres", axis=1)

در اینجا $dataframe$ نهایی فیلم‌ها آمده است:

In [166]:
movies_df.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


<br>


بعد، بیایید به $dataframe$ امتیازها نگاه کنیم

In [167]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


هر سطر در $dataframe$ امتیازها دارای یک شناسه کاربر مرتبط با حداقل یک فیلم، یک امتیاز و یک زمان‌نگار $(timestamp)$ است که نشان می‌دهد چه زمانی آن را بررسی کرده‌اند. ما به ستون زمان‌نگار نیازی نخواهیم داشت، بنابراین بیایید آن را حذف کنیم تا حافظه ذخیره شود

In [168]:
# یک سطر یا ستون مشخص را از یک دیتافریم حذف می‌کند Drop
ratings_df = ratings_df.drop("timestamp", axis=1)

در اینجا مشخصات دیتافریم نهایی امتیازها آورده شده است

In [169]:
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


<hr>

<a id="ref3"></a>

# Collaborative Filtering
### فیلتر کردن مشارکتی


حالا وقت آن است که کار خود را بر روی سیستم‌های پیشنهاددهنده آغاز کنیم

اولین تکنیکی که به آن نگاهی خواهیم انداخت، **فیلتر کردن مشارکتی** نام دارد که به عنوان **فیلتر کردن کاربر-کاربر** نیز شناخته می‌شود. همانطور که از نام جایگزین آن پیداست، این تکنیک از سایر کاربران برای توصیه آیتم‌ها به کاربر ورودی استفاده می‌کند. این روش سعی می‌کند کاربرانی را بیابد که ترجیحات و نظرات مشابهی با کاربر ورودی دارند و سپس آیتم‌هایی را که آن‌ها دوست داشته‌اند به کاربر ورودی توصیه می‌کند. روش‌های مختلفی برای یافتن کاربران مشابه وجود دارد (حتی برخی از آن‌ها از یادگیری ماشین استفاده می‌کنند) و روشی که ما در اینجا استفاده خواهیم کرد بر اساس **تابع همبستگی پیرسون** خواهد بود.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/images/User_Item.png" width=800px>

فرآیند ایجاد یک سیستم پیشنهاددهنده مبتنی بر کاربر به شرح زیر است:

*   یک کاربر با فیلم‌هایی که تماشا کرده است انتخاب کنید
*   بر اساس امتیاز او به فیلم‌ها، $X$ همسایه برتر را بیابید
*   سابقه فیلم تماشا شده توسط کاربر را برای هر همسایه دریافت کنید
*   با استفاده از یک فرمول، یک امتیاز شباهت محاسبه کنید
*   آیتم‌هایی با بالاترین امتیاز را توصیه کنید

بیایید با ایجاد یک کاربر ورودی برای توصیه فیلم شروع کنیم

توجه: برای افزودن فیلم‌های بیشتر، کافی است تعداد عناصر در $userInput$ را افزایش دهید

با خیال راحت فیلم‌های بیشتری اضافه کنید

$"Matrix, The"$: فقط مطمئن شوید که آن را با حروف بزرگ بنویسید و اگر فیلمی با $"The"$ شروع می‌شود، مانند $"The Matrix"$، آن را به این صورت بنویسید

In [170]:
userInput = [
            {"title":"Breakfast Club, The", "rating":5},
            {"title":"Toy Story", "rating":3.5},
            {"title":"Jumanji", "rating":2},
            {"title":"Pulp Fiction", "rating":5},
            {"title":"Akira", "rating":4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


#### افزودن $movieId$ به کاربر ورودی

با تکمیل ورودی، بیایید شناسه‌های فیلم‌های ورودی را از $dataframe$ فیلم‌ها استخراج کرده و آن‌ها را به آن اضافه کنیم

ما می‌توانیم این کار را با فیلتر کردن ابتدایی سطرهایی که حاوی عنوان فیلم‌های ورودی هستند و سپس ادغام این زیرمجموعه با $dataframe$ ورودی انجام دهیم. همچنین برای صرفه‌جویی در فضای حافظه، ستون‌های غیرضروری را برای ورودی حذف می‌کنیم

In [171]:
# فیلتر کردن فیلم‌ها بر اساس عنوان
inputId = movies_df[movies_df["title"].isin(inputMovies["title"].tolist())]
# سپس ادغام آن. این ادغام به صورت ضمنی بر اساس عنوان انجام می‌شود
inputMovies = pd.merge(inputId, inputMovies)

inputMovies = inputMovies.drop("year", axis=1)
# dataframe ورودی نهایی
# اگر فیلمی که در بالا اضافه کردید اینجا نیست، ممکن است در دیتافریم اصلی وجود نداشته باشد یا به شکل متفاوتی نوشته شده باشد، لطفاً بزرگ‌نویسی را بررسی کنید
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


#### کاربرانی که همان فیلم‌ها را دیده‌اند

حالا با داشتن شناسه‌های فیلم در ورودی خود، می‌توانیم زیرمجموعه‌ای از کاربرانی که فیلم‌های ورودی ما را تماشا کرده و بررسی کرده‌اند، به دست آوریم

In [172]:
# فیلتر کردن کاربرانی که فیلم‌هایی را تماشا کرده‌اند که کاربر ورودی تماشا کرده و ذخیره آن
userSubset = ratings_df[ratings_df["movieId"].isin(inputMovies["movieId"].tolist())]
userSubset.head()

,userId,movieId,rating
19,4,296,4.0
441,12,1968,3.0
479,13,2,2.0
531,13,1274,5.0
681,14,296,2.0


حالا سطرها را بر اساس شناسه کاربر گروه‌بندی می‌کنیم

In [173]:
userSubsetGroup = userSubset.groupby(["userId"])

In [174]:
type(list(userSubsetGroup.groups.keys())[0])

C:\Users\mrb\AppData\Local\Temp\ipykernel_12616\4023138889.py:1: Pandas4Warning: In a future version, the keys of `groups` will be a tuple with a single element, e.g. (userId,) , instead of a scalar, e.g. userId, when grouping by a list with a single element. Use ``df.groupby(by='a').groups`` instead of ``df.groupby(by=['a']).groups`` to avoid this warning
  type(list(userSubsetGroup.groups.keys())[0])


int

بیایید نگاهی به یکی از کاربران بیندازیم، برای مثال کاربری با $userID=1130$

In [175]:
# userSubsetGroup.get_group(4)

for uid, c in userSubsetGroup:
    if uid == 1130:
        display(c)
        break
c

,userId,movieId,rating
22884355,247753,1,3.0


علاوه بر این، این گروه‌ها را طوری مرتب می‌کنیم که کاربرانی که بیشترین تعداد فیلم مشترک با کاربر ورودی دارند در اولویت بالاتری قرار بگیرند. این کار توصیه‌های غنی‌تری ارائه می‌دهد، زیرا نیازی به بررسی تمام کاربران نخواهد بود

In [176]:
# مرتب‌سازی طوری که کاربران با بیشترین تعداد فیلم مشترک با ورودی در اولویت باشند
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

حالا بیایید به اولین کاربر نگاه کنیم

In [177]:
userSubsetGroup[0:3]

[((75,),
        userId  movieId  rating
  7507      75        1     5.0
  7508      75        2     3.5
  7540      75      296     5.0
  7633      75     1274     4.5
  7673      75     1968     5.0),
 ((106,),
        userId  movieId  rating
  9083     106        1     2.5
  9084     106        2     3.0
  9115     106      296     3.5
  9198     106     1274     3.0
  9238     106     1968     3.5),
 ((686,),
         userId  movieId  rating
  61336     686        1     4.0
  61337     686        2     3.0
  61377     686      296     4.0
  61478     686     1274     4.0
  61569     686     1968     5.0)]

#### شباهت کاربران به کاربر ورودی

در مرحله بعد، قصد داریم تمام کاربران (البته نه واقعاً همه!) را با کاربر مشخص‌شده خود مقایسه کنیم و کاربری را بیابیم که بیشترین شباهت را دارد
می‌خواهیم با استفاده از **ضریب همبستگی پیرسون** میزان شباهت هر کاربر را به کاربر ورودی محاسبه کنیم

این ضریب برای سنجش قدرت رابطه خطی بین دو متغیر به کار می‌رود. فرمول محاسبه آن بین دو مجموعه $X$ و $Y$ با $N$ مقدار در تصویر زیر قابل مشاهده است

**چرا ضریب همبستگی پیرسون؟**

ضریب پیرسون به تغییر مقیاس ناوردا است، یعنی ضرب همه عناصر در یک ثابت غیرصفر یا اضافه کردن یک ثابت به همه عناصر، مقدار آن را تغییر نمی‌دهد

برای مثال، اگر دو بردار $X$ و $Y$ داشته باشید، آنگاه

`pearson(X, Y) == pearson(X, 2 * Y + 3)`

این ویژگی در سیستم‌های پیشنهاد‌دهنده بسیار مهم است، زیرا ممکن است دو کاربر، دو سری آیتم را از نظر امتیازات مطلق کاملاً متفاوت ارزیابی کنند، اما در عین حال کاربران مشابهی باشند (یعنی سلیقه مشابهی داشته باشند) و الگوی امتیازدهی آن‌ها در مقیاس‌های مختلف یکسان باشد

مقادیر حاصل از این فرمول بین $`r = -1`$ تا $`r = 1`$ متغیر است
- عدد $`1`$ نشان‌دهنده همبستگی کامل مثبت بین دو موجودیت است
- عدد $`-1`$ نشان‌دهنده همبستگی کامل منفی است

در مورد ما، $`1`$ به معنای تشابه سلیقه دو کاربر است و $`-1`$ معنای عکس آن را دارد

یک زیرمجموعه از کاربران را برای پیمایش انتخاب می‌کنیم. این محدودیت به این دلیل اعمال می‌شود که نمی‌خواهیم زمان زیادی را برای بررسی تک‌تک کاربران هدر دهیم

In [178]:
userSubsetGroup = userSubsetGroup[0:100]

حالا ضریب همبستگی پیرسون بین کاربر ورودی و زیرمجموعه کاربران را محاسبه کرده و آن را در یک دیکشنری ذخیره می‌کنیم، که کلید آن شناسه کاربر و مقدار آن ضریب همبستگی است

In [179]:
# ذخیره ضریب همبستگی پیرسون در یک دیکشنری، که کلید آن شناسه کاربر و مقدار آن ضریب است
pearsonCorrelationDict = {}

# برای هر گروه کاربری در زیرمجموعه‌مان
for name, group in userSubsetGroup:

    # بیایید با مرتب‌سازی کاربر ورودی و گروه کاربری فعلی شروع کنیم تا مقادیر بعداً با هم مخلوط نشوند
    group = group.sort_values(by="movieId")
    inputMovies = inputMovies.sort_values(by="movieId")
    # را برای فرمول به دست آورید N مقدار
    nRatings = len(group)
    # امتیازهای بررسی فیلم‌هایی که هر دو کاربر در آن‌ها مشترک هستند را دریافت کنید
    temp_df = inputMovies[inputMovies["movieId"].isin(group["movieId"].tolist())]
    # سپس آن‌ها را در یک متغیر بافر موقت به صورت لیست ذخیره کنید تا محاسبات بعدی تسهیل شوند
    tempRatingList = temp_df["rating"].tolist()
    # همچنین بررسی‌های گروه کاربری فعلی را به صورت لیست قرار می‌دهیم
    tempGroupList = group["rating"].tolist()
    # حالا بیایید همبستگی پیرسون بین دو کاربر را محاسبه کنیم
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    # اگر مخرج مخالف صفر باشد، تقسیم می‌کنیم، در غیر این صورت، همبستگی صفر است
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


In [180]:
pearsonCorrelationDict.items()

dict_items([((75,), 0.8272781516947562), ((106,), 0.5860090386731182), ((686,), 0.8320502943378437), ((815,), 0.5765566601970551), ((1040,), 0.9434563530497265), ((1130,), 0.2891574659831201), ((1502,), 0.8770580193070299), ((1599,), 0.4385290096535153), ((1625,), 0.716114874039432), ((1950,), 0.179028718509858), ((2065,), 0.4385290096535153), ((2128,), 0.5860090386731196), ((2432,), 0.1386750490563073), ((2791,), 0.8770580193070299), ((2839,), 0.8204126541423674), ((2948,), -0.11720180773462392), ((3025,), 0.45124262819713973), ((3040,), 0.89514359254929), ((3186,), 0.6784622064861935), ((3271,), 0.26989594817970664), ((3429,), 0.0), ((3734,), -0.15041420939904673), ((4099,), 0.05860090386731196), ((4208,), 0.29417420270727607), ((4282,), -0.4385290096535115), ((4292,), 0.6564386345361464), ((4415,), -0.11183835382312353), ((4586,), -0.9024852563942795), ((4725,), -0.08006407690254357), ((4818,), 0.4885967564883424), ((5104,), 0.7674257668936507), ((5165,), -0.4385290096535153), ((554

In [181]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient="index")
pearsonDF.columns = ["similarityIndex"]
pearsonDF["userId"] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.827278,"(75,)"
1,0.586009,"(106,)"
2,0.832050,"(686,)"
3,0.576557,"(815,)"
4,0.943456,"(1040,)"


#### $x$ کاربر مشابه‌تر به کاربر ورودی

حالا بیایید ۵۰ کاربری را که بیشترین شباهت را به کاربر ورودی دارند، انتخاب کنیم

In [182]:
topUsers=pearsonDF.sort_values(by="similarityIndex", ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
64,0.961678,"(12325,)"
34,0.961538,"(6207,)"
55,0.961538,"(10707,)"
67,0.960769,"(13053,)"
4,0.943456,"(1040,)"


حالا بیایید شروع به پیشنهاد فیلم به کاربر ورودی کنیم

#### امتیازدهی کاربران انتخاب‌شده به همه فیلم‌ها

برای این کار، میانگین وزنی امتیازهای فیلم‌ها را با استفاده از ضریب همبستگی پیرسون به عنوان وزن محاسبه می‌کنیم. اما پیش از آن، باید فیلم‌های تماشاشده توسط کاربران موجود در $pearsonDF$ را از دیتافریم امتیازها استخراج کرده و سپس ضریب همبستگی آن‌ها را در یک ستون جدید به نام $"\_similarityIndex"$ ذخیره کنیم. این کار از طریق ادغام این دو جدول در زیر انجام می‌شود

In [183]:
topUsers["userId"] = topUsers["userId"].apply(lambda x: x[0] if isinstance(x, tuple) else x)

topUsers["userId"] = pd.to_numeric(topUsers["userId"], errors="coerce")
topUsers = topUsers.dropna(subset=["userId"])
topUsers["userId"] = topUsers["userId"].astype(int)

ratings_df["userId"] = ratings_df["userId"].astype(int)

In [184]:
topUsersRating=topUsers.merge(ratings_df, left_on="userId", right_on="userId", how="inner")
topUsersRating

,similarityIndex,userId,movieId,rating
0,0.961678,12325,1,3.5
1,0.961678,12325,2,1.5
2,0.961678,12325,3,3.0
3,0.961678,12325,5,0.5
4,0.961678,12325,6,2.5
...,...,...,...,...
47235,0.576557,815,146350,2.5
47236,0.576557,815,146656,3.5
47237,0.576557,815,148238,2.5
47238,0.576557,815,148626,3.5


حالا تنها کاری که باید انجام دهیم این است که امتیاز هر فیلم را در وزن آن (شاخص شباهت) ضرب کنیم، سپس امتیازهای جدید را جمع زده و آن را بر مجموع وزن‌ها تقسیم کنیم

این کار را به سادگی با ضرب دو ستون انجام می‌دهیم، سپس دیتافریم را بر اساس $movieId$ گروه‌بندی کرده و دو ستون را بر هم تقسیم می‌کنیم

این، ایده کلی کاربران مشابه به همراه فیلم‌های کاندید برای کاربر ورودی را نشان می‌دهد

In [185]:
# ضریب شباهت را در امتیازهای کاربر ضرب می‌کند
topUsersRating["weightedRating"] = topUsersRating["similarityIndex"]*topUsersRating["rating"]
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,0.961678,12325,1,3.5,3.365874
1,0.961678,12325,2,1.5,1.442517
2,0.961678,12325,3,3.0,2.885035
3,0.961678,12325,5,0.5,0.480839
4,0.961678,12325,6,2.5,2.404196


In [186]:
# پس از گروه‌بندی بر اساس شناسه کاربر، یک جمع روی آن اعمال می‌کند
tempTopUsersRating = topUsersRating.groupby("movieId").sum()[["similarityIndex","weightedRating"]]
tempTopUsersRating.columns = ["sum_similarityIndex","sum_weightedRating"]
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,38.376281,140.800834
2,38.376281,96.656745
3,10.253981,27.254477
4,0.929294,2.787882
5,11.723262,27.151751


In [187]:
# یک دیتافریم خالی ایجاد می‌کند
recommendation_df = pd.DataFrame()
# حالا میانگین وزنی را محاسبه می‌کنیم
recommendation_df["weighted average recommendation score"] = tempTopUsersRating["sum_weightedRating"]/tempTopUsersRating["sum_similarityIndex"]
recommendation_df["movieId"] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.668955,1
2,2.518658,2
3,2.657941,3
4,3.000000,4
5,2.316058,5


حال بیایید آن را مرتب کنیم و 20 فیلم برتری را که الگوریتم توصیه کرده است ببینیم

In [188]:
recommendation_df = recommendation_df.sort_values(by="weighted average recommendation score", ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
3329,5.0,3329
2284,5.0,2284
5073,5.0,5073
28,5.0,28
2848,5.0,2848
136485,5.0,136485
4252,5.0,4252
99,5.0,99
121,5.0,121


In [189]:
movies_df.loc[movies_df["movieId"].isin(recommendation_df.head(10)["movieId"].tolist())]

,movieId,title,year
27,28,Persuasion,1995
97,99,Heidi Fleiss: Hollywood Madam,1995
119,121,"Boys of St. Vincent, The",1992
2200,2284,Bandit Queen,1994
2763,2848,Othello (Tragedy of Othello: The Moor of Venic...,1952
3243,3329,"Year My Voice Broke, The",1987
4159,4252,"Circle, The (Dayereh)",2000
4978,5073,"Son's Room, The (Stanza del figlio, La)",2001
8653,26158,Closely Watched Trains (Ostre sledované vlaky),1966
30005,136485,Robot Chicken: Star Wars,2007


### مزایا و معایب فیلتر کردن مشارکتی

##### مزایا

*   امتیازات کاربران دیگر را در نظر می‌گیرد
*   نیاز به مطالعه یا استخراج اطلاعات از آیتم توصیه شده ندارد
*   با علایق کاربر که ممکن است با گذشت زمان تغییر کند سازگار می‌شود

##### معایب

*   تابع تقریب می‌تواند کند باشد
*   ممکن است تعداد کاربران کافی برای تقریب وجود نداشته باشد
*   مسائل مربوط به حریم خصوصی هنگام تلاش برای یادگیری ترجیحات کاربر


<h2>می‌خواهید بیشتر یاد بگیرید؟</h2>

پلتفرم **هوکو** یک بستر جامع تحلیلی و هوش مصنوعی است که مجموعه‌ای از الگوریتم‌های یادگیری ماشین، ابزارهای تحلیل داده و راهکارهای پیش‌بینی هوشمند را در اختیار شما قرار می‌دهد.
این پلتفرم به شما کمک می‌کند تا تصمیم‌های دقیق‌تر، سریع‌تر و مبتنی بر داده بگیرید؛ 

چه به‌صورت فردی، چه در سطح تیمی و یا در مقیاس سازمانی

___

اکنون می‌توانید نسخه آزمایشی رایگان هوکو را فعال کرده و قدرت هوش مصنوعی را در تصمیم‌گیری‌های خود تجربه کنید

 <a href="https://hooko.ir">HOOKO.ir شروع تجربه در </a>

 ___

## Author

Mahdi Fatemi (HAKAN)

Instagram: @Fatemi_303

09220630140

## web
www.hooko.ir

## Change Log

|  Date (YYYY-MM-DD) |  Version | Changed By  |  Change Description |
|---|---|---|---|
| 2025-11-06 | 1.0  | HAKAN Fatemi  |  ... |

## <h3 align="center"> © HOOKO.IR Corporation. All rights reserved. <h3/>
